In [2]:
import http.client
import json
import csv
from datetime import datetime
import html
import re

In [2]:
occupations_list = [
    'ICT ales professional',
    'Marketing manager',
    'Product analyst',
    'Product manager',
    'Product designer',
    'Business Intelligence professional',
    'Infraestructure engineer',
    'Computer ystems Analyst',
    'Software infrastructure architect',
    'Web developer',
    'Software developer',
    'App developer',
    'User interface designer',
    'Software engineer',
    'Software architect',
    'Software Quality Assurance Analysts and Testers',
    'Embedded ystems engineer',
    'Web and Digital Interface Designers',
    'Database infrastructure engineer',
    'Network architect',
    'Database Administrators',
    'Database Architects',
    'Network and Computer ystems Administrator',
    'Artificial intelligence',
    'Machine Learning engineer',
    'Data cience engineer',
    'Data analyst',
    'Data cientist',
    'Artifical Intelligence cientist',
    'Data architect',
    'ICT ecurity pecialist',
    'IT ecurity operations',
    'Information ecurity Analysts',
    'Product ecurity and IT ecurity Integration pecialist',
    'Product risk pecialist',
    'ICT ecurity pecialist',
    'Security architect',
    'Database support engineer',
    'Data center operations engineer',
    'Support systems engineer',
    'Computer Network upport pecialists'
    ]

In [22]:
# Configuración de conexión y headers de la API
conn = http.client.HTTPSConnection("indeed12.p.rapidapi.com")
headers = {
    'X-RapidAPI-Key': "7aaca348d5msh9e827a8093c9079p1cebe3jsn33ba8196246e",
    'X-RapidAPI-Host': "indeed12.p.rapidapi.com"
}

In [23]:
# Función para obtener los IDs de trabajos
def get_job_ids(query, location, page_id):
    conn.request("GET", f"/jobs/search?query={query}&location={location}&page_id={page_id}&radius=50", headers=headers)
    res = conn.getresponse()
    print(res)
    data = res.read()
    jobs_data = json.loads(data.decode("utf-8"))
    job_ids = [job['id'] for job in jobs_data['hits']]
    return job_ids

In [5]:
# Función para obtener los detalles de un trabajo
def get_job_details(job_id):
    conn.request("GET", f"/job/{job_id}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    job_details = json.loads(data.decode("utf-8"))
    return job_details

In [6]:
def remove_html_tags_and_clean(text):
    # Elimina cualquier tag HTML
    clean_text = re.sub(r'<[^>]+>', '', text)
    # Convierte entidades HTML a texto
    clean_text = html.unescape(clean_text)
    # Elimina espacios extra y saltos de línea múltiples
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

In [24]:
# Obtener IDs de trabajos
job_ids = get_job_ids("Database+support+engineer", "Mexico", 1)

In [25]:
# Crear archivo CSV y escribir los detalles de los trabajos
with open('indeed_rapidapi.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Escribir cabecera del CSV
    writer.writerow(['JobID', 'JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Details', 'Salary', 'JobUrl'])
    
    # Recuperar y escribir detalles de cada trabajo
    for job_id in job_ids:
        details = get_job_details(job_id)
        job_title = details['job_title']
        company = details['company']['name']
        location = details['location']
        post_date = details['creation_date']
        extract_date = datetime.now().strftime('%Y-%m-%d')
        full_details = remove_html_tags_and_clean(details['description'])  # Elimina HTML de los detalles completos
        salary_range = f"${details['salary']['min']:,} - ${details['salary']['max']:,} per year" if details.get('salary') else "Not provided"
        job_url = details['indeed_final_url']
        
        # Escribir fila en CSV
        writer.writerow([job_id, job_title, company, location, post_date, extract_date, full_details, salary_range, job_url])

print("Los detalles de los trabajos han sido guardados en 'indeed_rapidapi.csv'")


Los detalles de los trabajos han sido guardados en 'job_details.csv'


In [ ]:
# Si hay trabajos disponibles, obtener y mostrar los detalles del primer trabajo
if job_ids:
    job_details = get_job_details(job_ids[0])
    print(json.dumps(job_details, indent=0))
else:
    print("No se encontraron trabajos con los criterios especificados.")